# Regional QC

This notebook outputs some plots and QC results following inference for a given country. 

In [ ]:
region='cri'
country='costarica'

## General file and notebook organisation

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import boto3
import json
from boto3.s3.transfer import TransferConfig
from PIL import Image
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta, time
import matplotlib.dates as mdates
import numpy as np
from matplotlib.colors import LogNorm, ListedColormap
from tqdm import tqdm
import math
import matplotlib as mpl

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
# Publication-ready plot settings
mpl.rcParams.update({
    'font.size': 16,           # Larger base font size
    'axes.titlesize': 18,     # Title font size
    'axes.labelsize': 16,     # Axis label font size
    'xtick.labelsize': 14,    # Tick label font size
    'ytick.labelsize': 14,    # Tick label font size
    'legend.fontsize': 14,    # Legend font size
    'figure.titlesize': 20,   # Figure title font size
    'figure.dpi': 300,        # High-res output
    'savefig.dpi': 300,       # High-res saved figures
    'figure.autolayout': True # Tight layout
})

In [ ]:
# set the working directory
os.chdir(os.path.expanduser('~/amber-inferences'))

In [ ]:
# %pip install -e .

In [ ]:

download_dir=f'./data/qc_plots/{country}'
plot_dir = f'./sandbox/plots/{country}'
os.makedirs(download_dir, exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)


inference_dir = os.path.abspath(f'/gws/nopw/j04/ceh_generic/kgoldmann/{country}_inferences_tracking/')

#listdir recursively
def listdir_recursive(path):
    for root, dirs, files in os.walk(path):
        for file in files:
            yield os.path.join(root, file)

# Get all csv files in the inference directory
inference_csvs = list(listdir_recursive(inference_dir))
inference_csvs = [c for c in inference_csvs if c.endswith('.csv')]
inference_csvs = [c for c in inference_csvs if 'compute' not in c]

In [ ]:
len(inference_csvs)

## Plotting and Data Wrangling Functions

In [ ]:
def annotate_image(image_path, dir, df, ax, scaling_required=False, crop_to_highlight=None, buffer=5, subtitle=None):
    df_image = df.loc[df['image_path'] == image_path, ]

    img = plt.imread(f"{dir}/{os.path.basename(image_path)}")
    image = Image.open(f"{dir}/{os.path.basename(image_path)}").convert("RGB")
    # subp = ax.imshow(img, origin='lower')

    draw = ImageDraw.Draw(image)

    for j, row in df_image.iterrows():
        x_min = float(row['x_min']) -buffer
        y_min = float(row['y_min']) -buffer
        x_max = float(row['x_max']) +buffer
        y_max = float(row['y_max']) +buffer

        if scaling_required:
            original_width, original_height = image.size
            x_min = x_min /300 * original_width
            y_min = y_min /300 * original_height
            x_max = x_max /300 * original_width
            y_max = y_max /300 * original_height

        x = float(x_min)
        y = float(y_min)
        w = float(x_max - x_min)
        h = float(y_max - y_min)

        if (row['class_name'] == "moth") and ('Lepidoptera' not in row['order_name']):
            col = 'orange'
        elif (row['class_name'] != "moth") and ('Lepidoptera' in row['order_name']):
            col = 'purple'
        elif (row['class_name'] == "moth") and ('Lepidoptera' in row['order_name']):
                col = 'green'
        else:
            col = 'red'

        alph = 1
        if crop_to_highlight is not None:
            if row['crop_status'] != crop_to_highlight:
                alph = 0.2

        try:
            if os.path.exists("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"):
                font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
            else:
                import cv2

                font_path = os.path.join(
                    cv2.__path__[0], "qt", "fonts", "DejaVuSans.ttf"
                )
            font = ImageFont.truetype(font_path, size=50)
        except Exception as e:
            print(f"Loading default font, could not another: {e}")
            font = ImageFont.load_default()


        if (row['class_name'] == "moth") or ("Lepidoptera" in row['order_name']):
            draw.text(
                (x_min, y_min),
                f"{row['top_1_species']}: {row['top_1_confidence']:.2f}",
                fill=col,
                font=font,
            )

        draw.rectangle([x_min, y_min, x_max, y_max], outline=col, width=3)

        # rect = plt.Rectangle((x, y), w, h, fill=False, edgecolor=col, linewidth=1, alpha=alph)
        # ax.add_patch(rect)

    ax = plt.axes()

    ax.add_patch(img)
    if not subtitle:
        subtitle=f"{os.path.basename(image_path)}"
    ax.set_title(subtitle)
    ax.axis('off')
    # return subp

In [ ]:
def download_images(s3_client, config, key, download_dir, bucket_name):
    download_path = os.path.join(download_dir, os.path.basename(key))
    s3_client.download_file(bucket_name, key, download_path, Config=config)

In [ ]:
def initialise_session(credentials_file="credentials.json"):
    """
    Load AWS and API credentials from a configuration file and initialise an AWS session.

    Args:
        credentials_file (str): Path to the credentials JSON file.

    Returns:
        boto3.Client: Initialised S3 client.
    """
    with open(credentials_file, encoding="utf-8") as config_file:
        aws_credentials = json.load(config_file)
    session = boto3.Session(
        aws_access_key_id=aws_credentials["AWS_ACCESS_KEY_ID"],
        aws_secret_access_key=aws_credentials["AWS_SECRET_ACCESS_KEY"],
        region_name=aws_credentials["AWS_REGION"],
    )
    client = session.client("s3", endpoint_url=aws_credentials["AWS_URL_ENDPOINT"])
    return client

client = initialise_session('./credentials.json')

In [ ]:
# Transfer configuration for optimised S3 download
transfer_config = TransferConfig(
    max_concurrency=20,  # Increase the number of concurrent transfers
    multipart_threshold=8 * 1024 * 1024,  # 8MB
    max_io_queue=1000,
    io_chunksize=262144,  # 256KB
)

In [ ]:
inference_csvs[0]

In [ ]:
def moth_only_df(inference_csvs):
    df = pd.DataFrame()
    for c in tqdm(inference_csvs, desc='Reading in the csvs'):
        try:
            input_df = pd.read_csv(c, low_memory=False)
            input_df = input_df.loc[input_df['crop_status'] != 'NO DETECTIONS FOR IMAGE', ]
            input_df = input_df.drop_duplicates(subset=[ 'x_min', 'x_max', 'y_min', 'y_max'])

            input_df = input_df.loc[input_df['top_1_species'].isna() == False, ]
            # if deployment_name in columns set dep=deployment_name, else split on basename
            if 'deployment_name' in input_df.columns:
                input_df['dep'] = input_df['deployment_name']
            else:
                input_df['dep'] = os.path.basename(c).split('.')[0].split('_')[0]
            input_df['crop_area'] = (input_df['x_max'] - input_df['x_min']) * (input_df['y_max'] - input_df['y_min'])

            # set new keys column as 'dep' and 'image_path' combined
            input_df['keys'] = input_df['image_path'].apply(lambda x: f"{input_df['deployment_id'].iloc[0]}/snapshot_images/{os.path.basename(x)}")
            df = pd.concat([df, input_df])
            del input_df
        except Exception as e:
            print(f" - Error reading {c}: {e}")
            continue

    return df

In [ ]:
def cat_summary(inference_csvs, category='order_name', macro_only=False):
    df = pd.DataFrame()
    for c in inference_csvs:
        try:
            input_df = pd.read_csv(c, low_memory=False)
        except Exception as e:
            print(f" - Error reading {c}: {e}")
            continue
        input_df = input_df.loc[input_df['crop_status'] != 'NO DETECTIONS FOR IMAGE', ]
        input_df = input_df.drop_duplicates(subset=[ 'x_min', 'x_max', 'y_min', 'y_max'])

        # if deployment_name in columns set dep=deployment_name, else split on basename
        if 'deployment_name' in input_df.columns:
            input_df['dep'] = input_df['deployment_name']
        else:
            input_df['dep'] = os.path.basename(c).split('.')[0].split('_')[0]
        input_df['crop_area'] = (input_df['x_max'] - input_df['x_min']) * (input_df['y_max'] - input_df['y_min'])

        if macro_only:
            input_df = input_df.loc[input_df['order_name'] == 'Lepidoptera Macros', ]

        # summarise the order_name by deployment
        summary = input_df[['dep', category]].value_counts()
        summary = summary.reset_index()
        summary.columns = ['deployment', category, 'count']
        summary['file'] = os.path.basename(c)

        df = pd.concat([df, summary], ignore_index=True)
        del input_df

    df = df[['deployment', category, 'count']].groupby(['deployment', category]).sum().reset_index()

    df = df.sort_values(by=['deployment', 'count'], ascending=[True, False])
    return df

In [ ]:
def get_date(file_name, format="%Y-%m-%d"):
    """
    Extracts the date from the file name based on the specified format.
    Assuming the date is in the filename and formatted as YYYYMMDD or similar.
    """
    try:
        file_name = os.path.splitext(file_name)[0]
        file_raw = os.path.basename(file_name).replace("_", "-").split("-")
        file = [x for x in file_raw if x.startswith("202")][0]

        # catch for delim between date and time in file name
        if len(file) < 12:
            i0 = [idx for idx in range(len(file_raw)) if file_raw[idx].startswith("202")][0]
            file = ('').join(file_raw[i0:i0+2])

        image_dt = datetime.strptime(file, "%Y%m%d%H%M%S%f")
        image_dt = datetime.strftime(image_dt, format)
        return image_dt

    except Exception as e:
        print(f"  - Error parsing date from file name {file_name}: {e}")
        return np.nan

def assign_night(ts, night_endpoint=12, night_startpoint=12):
    """
    Defines the recording night from date and time.
    The recording night cutoff is defined between night_startpoint on day 1
    and night_endpoint day 2.
    """
    try:
        if ts.time() < time(night_endpoint, 0):  # before night_endpoint o'clock
            night_start = ts.date() - timedelta(days=1)
        elif ts.time() >= time(night_startpoint, 0):  # after night_startpoint or later
            night_start = ts.date()
        else:
            # times not included in this overnight window
            night_start = pd.NaT
        if pd.isna(night_start):
            return None
        night_end = night_start + timedelta(days=1)
        return night_start
    except Exception as e:
        return 'No known date'

In [ ]:
def activity_plot_data(df, drop_empty_days = True, min_date=pd.to_datetime('2024-01-01'), end_date=None, count_column='count'):
    df['session'] = pd.to_datetime(df['session'])

    if end_date is None:
        end_date = df['session'].max()

    all_dates = pd.date_range(start=min_date, end=end_date)
    df = df.set_index('session').fillna(0).reindex(all_dates).rename_axis('session').reset_index()

    # Add week, day of week, and year
    df['week'] = df['session'].dt.isocalendar().week
    df['weekday'] = df['session'].dt.weekday  # Monday=0
    df['month'] = df['session'].dt.month
    df['year'] = df['session'].dt.year
    df['week_number'] = ((df['session'] - min_date).dt.days // 7).astype(int)
    df['week_label'] = 'W' + df['week_number'].astype(str)

    # Some dates in last week of December may belong to week 1 of next year
    df.loc[df['week'] == 1, 'year'] = df['session'].dt.year

    # Pivot for heatmap
    heatmap_data = df.pivot_table(index='weekday', columns='week_number', values=count_column, aggfunc='sum')

    # Reorder to GitHub style
    day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    heatmap_data.index = [day_names[i] for i in heatmap_data.index]
    heatmap_data = heatmap_data.reindex(['Sun', 'Sat', 'Fri', 'Thu', 'Wed', 'Tue', 'Mon'])

    # Determine month and year labels
    unique_weeks = df.drop_duplicates('week_number')
    month_labels = unique_weeks.set_index('week_number').loc[heatmap_data.columns, 'session'].dt.strftime('%b')
    week_labels = unique_weeks.set_index('week_number').loc[heatmap_data.columns, 'week_label']
    year_labels = unique_weeks.set_index('week_number').loc[heatmap_data.columns, 'year']


    if drop_empty_days:
        heatmap_data = heatmap_data.fillna(0)
        heatmap_data = heatmap_data.loc[(heatmap_data != 0).any(axis=1)]
        heatmap_data = heatmap_data.dropna(axis=0, how='all')

    return [heatmap_data, month_labels, week_labels, year_labels]

In [ ]:
def activity_plot(df,
                  ax,
                  dep,
                  min_date=pd.to_datetime('2024-01-01'),
                  vmin=0,
                  vmax=1e6,
                  custom_cmap='Greens',
                  include_month_labels=True,
                  end_date=None,
                  label_buffer=1,
                  show_colourbar=True,
                  drop_empty_days=True,
                  count_column='count',
    ):

    heatmap_data, month_labels, week_labels, year_labels = activity_plot_data(df, drop_empty_days=drop_empty_days, min_date=min_date, end_date=end_date, count_column=count_column)
    masked_data = np.ma.masked_invalid(heatmap_data.values)

    norm = LogNorm(vmin=vmin, vmax=vmax + 1)

    c = ax.pcolor(masked_data, cmap=custom_cmap,
                  edgecolors='white', linewidths=1, norm=norm)

    ax.set_yticks(np.arange(0.5, len(heatmap_data.index), 1))
    ax.set_yticklabels(heatmap_data.index)

    ax.set_xticks(np.arange(0.5, len(week_labels), 1))
    ax.set_xticklabels(week_labels, rotation=90)

    if include_month_labels:
        for i, label in enumerate(month_labels):
            if i > 0 and month_labels[i] != month_labels[i - 1]:
                ax.text(i + 0.5, -2*label_buffer, label, ha='center', va='center')
            elif i == 0:
                ax.text(i + 0.5, -2*label_buffer, label, ha='center', va='center')

        for i, label in enumerate(year_labels):
            if i > 0 and year_labels[i] != year_labels[i - 1]:
                ax.text(i + 0.5, -3*label_buffer, label, ha='center', va='center')
            elif i == 0:
                ax.text(i + 0.5, -3*label_buffer, label, ha='center', va='center')

    if show_colourbar:
        fig.colorbar(c, ax=ax, orientation='vertical', label='Number of crops')
    ax.set_title(dep)

    return c

# Activity Plots

## Data Wrangling

In [ ]:
ami_names = {'BAMK': 'Bishan-Ang Mo Kio Park',
    'BTNR': 'Bukit Timah Nature Reserve',
    'JLG': 'Jurong Lake Gardens',
    'NTU': 'Nanyang Technological University',
    'SBWR': 'Sungei Buloh Wetland Reserve',
    'W02': 'Central Catchment Nature Reserve',
    'SBG': 'Singapore Botanic Gardens',
    'RCHG': 'Rail Corridor Holland Green'
}

In [ ]:
# Function to assign index
def recording_index(hour):
    if 13 <= hour <= 23:
        return hour - 13 + 1
    elif 0 <= hour <= 12:
        return hour + 12
    else:
        return None           # Out of the range

In [ ]:
# create a df for deployment counts
dep_session = pd.DataFrame()
dep_session_moth = pd.DataFrame()
dep_hour = pd.DataFrame()
for c in tqdm(inference_csvs, desc='reading in the csvs'):
    try:
        input_df = pd.read_csv(c, low_memory=False)
        input_df = input_df.loc[input_df['crop_status'] != 'No detections for this image.', ]
        input_df = input_df.loc[input_df['crop_status'] != 'Image corrupt', ]
    except Exception as e:
        print(f" - Error reading {c}: {e}")
        continue


    if input_df.shape[0] == 0:
        print(f"  - No detections in {os.path.basename(c)}")
        continue
    input_df = input_df.drop_duplicates(subset=[ 'x_min', 'x_max', 'y_min', 'y_max'])


    # if deployment_name in columns set dep=deployment_name, else split on basename
    if 'deployment_name' in input_df.columns:
        input_df['dep'] = input_df['deployment_name']
    else:
        input_df['dep'] = os.path.basename(c).split('.')[0].split('_')[0]

    # rename
    for name, ami_name in ami_names.items():
        input_df.loc[input_df['dep'] == name, 'dep'] = ami_name

    input_df['crop_area'] = (input_df['x_max'] - input_df['x_min']) * (input_df['y_max'] - input_df['y_min'])
    input_df['datetime'] = input_df['image_path'].apply(lambda x: get_date(x, format="%Y-%m-%d %H:%M:%S"))
    input_df['datetime'] = pd.to_datetime(input_df['datetime'])
    input_df['hour'] = input_df['datetime'].dt.hour.astype(float)

    # add an column with index for hour, in order of appearance
    input_df['recording_hour'] = input_df['hour'].apply(recording_index)
    input_df['recording_hour'] = input_df['recording_hour'] - input_df['recording_hour'].min() + 1 # rescale

    input_df['hour_fraction'] = 1.0 # to represent whole hour covered by the time period

    # for first hour, change to fraction of hour corresponding to first timeframe
    start_time = 1 - input_df['datetime'].min().minute / 60
    end_time = input_df['datetime'].max().minute / 60
    if end_time == 0:
        end_time = 10/60/60
    input_df.loc[input_df['recording_hour'] == 1, 'hour_fraction'] = start_time
    input_df.loc[input_df['recording_hour'] == input_df['recording_hour'].max(), 'hour_fraction'] = end_time

    running_time = (input_df['datetime'].max() - input_df['datetime'].min()).total_seconds() / 3600

    # get the session night
    input_df['session'] = input_df['datetime'].apply(assign_night)

    # summarise the order_name by deployment
    summary = input_df[['dep', 'session']].value_counts()
    summary = summary.reset_index()
    summary['file'] = os.path.basename(c)
    summary['hourly_count'] = summary['count'] / running_time
    summary['hourly_count'] = summary['hourly_count'].replace([np.inf, -np.inf], 0)
    dep_session = pd.concat([dep_session, summary], ignore_index=True)

    summary = input_df[['dep', 'hour', 'hour_fraction', 'order_name']].value_counts()
    summary = summary.reset_index()
    summary['file'] = os.path.basename(c)
    summary['adjusted_count'] = summary['count'] / summary['hour_fraction']
    summary['adjusted_count'] = summary['adjusted_count'].replace([np.inf, -np.inf], np.nan)
    dep_hour = pd.concat([dep_hour, summary], ignore_index=True)

    # summarise the order_name by deployment
    input_df = input_df.loc[input_df['order_name'].isna() == False, ]
    try:
        input_df = input_df.loc[input_df['order_name'].str.contains("Lepidoptera"), ]
        summary_moth = input_df[['dep', 'session']].value_counts()
        summary_moth = summary_moth.reset_index()
        summary_moth['hourly_count'] = summary_moth['count'] / running_time
        summary_moth['hourly_count'] = summary_moth['hourly_count'].replace([np.inf, -np.inf], 0)

        summary_moth['file'] = os.path.basename(c)
        dep_session_moth = pd.concat([dep_session_moth, summary_moth], ignore_index=True)
    except Exception as e:
        print(f"  - Error processing moths in {c}: {e}")

    del input_df


In [ ]:
deployments = dep_session['dep'].unique()
print(deployments)

In [ ]:
dep_hour

In [ ]:
dep_hour_df = dep_hour[['dep', 'adjusted_count', 'count', 'hour', 'order_name']].groupby(['dep', 'hour', 'order_name']).mean().reset_index()
dep_hour_df

In [ ]:
dep_session_df = dep_session.loc[dep_session['session'] != 'No known date', ]
dep_session_df = dep_session_df[['dep', 'hourly_count', 'session']].groupby(['dep', 'session']).mean().reset_index()
dep_session_df.head()

In [ ]:
dep_session_all_moth = dep_session_moth.loc[dep_session_moth['session'] != 'No known date', ]
dep_session_all_moth = dep_session_all_moth[['hourly_count', 'session']].groupby(['session']).mean().reset_index()
dep_session_all_moth.head()

In [ ]:
dep_session_df_moth = dep_session_moth.loc[dep_session_moth['session'] != 'No known date', ]
dep_session_df_moth = dep_session_df_moth[['dep', 'hourly_count', 'session']].groupby(['dep', 'session']).mean().reset_index()
dep_session_df_moth.head()

In [ ]:
dep_df = dep_session_df[['dep', 'hourly_count']].groupby(['dep']).mean().reset_index()
dep_df = dep_df.sort_values(by=['dep', 'hourly_count'], ascending=[True, False])
dep_df.head()

In [ ]:
dep_session_df_all = dep_session_df[['session', 'hourly_count']].groupby(['session']).mean().reset_index()
dep_session_df_all.head()

## Github style activity plots

In [ ]:
cmap = plt.cm.Greens
newcolors = cmap(np.linspace(0, 1, 256))
newcolors[0] = [1, 1, 1, 1]  # RGBA for white
custom_cmap = ListedColormap(newcolors)
custom_cmap.set_bad(color=(0.85, 0.85, 0.85, 1))

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))

activity_plot(dep_session_df_all, ax, dep='All', min_date=pd.to_datetime('2024-01-01'),
              vmin=1, vmax=dep_session_df_all['hourly_count'].max(), custom_cmap=custom_cmap,
              label_buffer=1.5, count_column='hourly_count')

ax.set_title("Average Hourly Activity")
ax.set_aspect('equal')
fig.tight_layout(pad=1.0)
plt.show()

In [ ]:
min_date=pd.to_datetime('2024-01-01')
max_labels = ((pd.to_datetime(dep_session_df['session']).max() - min_date).days // 7)
date_range = pd.date_range(start=min_date, end=pd.to_datetime(dep_session_df['session']).max(), freq='W')

week_labels = ['W' + str(i+1) for i in range(len(date_range))]
year_labels = [x.strftime('%Y') for x in date_range]
month_labels = [x.strftime('%b') for x in date_range]

In [ ]:
# subplots with activity plots for each deployment
ncols=1
nrows=int(math.ceil(len(deployments)/ncols))
fig, axs = plt.subplots(nrows, ncols, figsize=(20, 20), sharex='all')
axs = axs.flatten()

for i, dep in enumerate(deployments):
    c=activity_plot(
        dep_session_df.loc[dep_session_df['dep'] == dep, ['dep', 'session', 'hourly_count']],
        axs[i], dep=dep, min_date=pd.to_datetime('2024-01-01'),
        vmin=1, vmax=dep_session_df['hourly_count'].max(), label_buffer=2,
        custom_cmap=custom_cmap, drop_empty_days=True, include_month_labels=False,
        end_date = dep_session_df['session'].max(), show_colourbar=False,
        count_column='hourly_count'
    )
    axs[i].set_aspect('equal')

axs[i].set_xticks(np.arange(0.5, len(week_labels), 1))
axs[i].set_xticklabels(week_labels, rotation=90)
axs[i-1].set_xticks(np.arange(0.5, len(week_labels), 1))
axs[i-1].set_xticklabels(week_labels, rotation=90)

cbar_ax = fig.add_axes([0.25, -0.1*ncols, 0.5, 0.03])  # [left, bottom, width, height]
fig.colorbar(c, cax=cbar_ax, orientation='horizontal', label='Average number of crops per hour (log scale)')

# Optional week index labels
for i, label in enumerate(month_labels):
    if i > 0 and month_labels[i] != month_labels[i - 1]:
        for j in range(1, ncols+1):
            axs[len(dep_df['dep'].unique())-j].text(i + 0.5, -3.5*ncols, label, ha='center', va='center')
    elif i == 0:
        for j in range(1, ncols+1):
            axs[len(dep_df['dep'].unique())-j].text(i + 0.5, -3.5*ncols, label, ha='center', va='center')


for i, label in enumerate(year_labels):
    if i > 0 and year_labels[i] != year_labels[i - 1]:
        for j in range(1, ncols+1):
            axs[len(dep_df['dep'].unique())-j].text(i + 0.5, -5*ncols, label, ha='center', va='center')
    elif i == 0:
        for j in range(1, ncols+1):
            axs[len(dep_df['dep'].unique())-j].text(i + 0.5, -5*ncols, label, ha='center', va='center')

plt.suptitle("Average Hourly Activity")
fig.tight_layout(pad=1.0)
plt.show()

# save the plot
fig.savefig(f"{plot_dir}/average_hourly_activity.png", bbox_inches='tight', dpi=300)

In [ ]:
cmap = plt.cm.Purples
newcolors = cmap(np.linspace(0, 1, 256))
newcolors[0] = [1, 1, 1, 1]  # RGBA for white
custom_cmap = ListedColormap(newcolors)
custom_cmap.set_bad(color=(0.85, 0.85, 0.85, 1))

In [ ]:
dep_session_moth.head()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 4))

activity_plot(dep_session_all_moth, ax, dep='All', min_date=pd.to_datetime('2024-01-01'),
              vmin=1, vmax=dep_session_all_moth['hourly_count'].max(),
              custom_cmap=custom_cmap, label_buffer=1.5, count_column='hourly_count')

ax.set_title("Crop Activity Heatmap")
ax.set_aspect('equal')
plt.show()

In [ ]:
# subplots with activity plots for each deployment
ncols=1
nrows=int(math.ceil(len(deployments)/ncols))
fig, axs = plt.subplots(nrows, ncols, figsize=(20, 25), sharex='all')
axs = axs.flatten()

for i, dep in enumerate(dep_df['dep'].unique()):
    c=activity_plot(dep_session_df_moth.loc[dep_session_df_moth['dep'] == dep, ['dep', 'session', 'hourly_count']],
                  axs[i], dep=dep, min_date=pd.to_datetime('2024-01-01'),
                vmin=1, vmax=dep_session_df_moth['hourly_count'].max(), label_buffer=2,
                custom_cmap=custom_cmap, include_month_labels=False,
                end_date = pd.to_datetime(dep_session_df_moth['session']).max(),
                    drop_empty_days=True,
                show_colourbar=False, count_column='hourly_count')
    axs[i].set_aspect('equal')

axs[i].set_xticks(np.arange(0.5, len(week_labels), 1))
axs[i].set_xticklabels(week_labels, rotation=90)
axs[i-1].set_xticks(np.arange(0.5, len(week_labels), 1))
axs[i-1].set_xticklabels(week_labels, rotation=90)

cbar_ax = fig.add_axes([0.25, -0.1*ncols, 0.5, 0.03])  # [left, bottom, width, height]
fig.colorbar(c, cax=cbar_ax, orientation='horizontal', label='Hourly moth count (log scale)')

# Optional week index labels
for i, label in enumerate(month_labels):
    if i > 0 and month_labels[i] != month_labels[i - 1]:
        for j in range(1, ncols+1):
            axs[len(dep_df['dep'].unique())-j].text(i + 0.5, -3.5*ncols, label, ha='center', va='center')
    elif i == 0:
        for j in range(1, ncols+1):
            axs[len(dep_df['dep'].unique())-j].text(i + 0.5, -3.5*ncols, label, ha='center', va='center')


for i, label in enumerate(year_labels):
    if i > 0 and year_labels[i] != year_labels[i - 1]:
        for j in range(1, ncols+1):
            axs[len(dep_df['dep'].unique())-j].text(i + 0.5, -5*ncols, label, ha='center', va='center')
    elif i == 0:
        for j in range(1, ncols+1):
            axs[len(dep_df['dep'].unique())-j].text(i + 0.5, -5*ncols, label, ha='center', va='center')

plt.suptitle("Moth Crop Activity Heatmap")

# save the plot
fig.savefig(f"{plot_dir}/moth_crop_activity_heatmap.png", bbox_inches='tight', dpi=300)

# Polar Plots

In [ ]:
dep_hour_df

In [ ]:
dep_hour_df_all = dep_hour_df[['hour',  'order_name', 'adjusted_count', 'count']].groupby(['hour', 'order_name']).mean().reset_index()

dep_hour_df_all['adjust'] = dep_hour_df_all['adjusted_count'] != dep_hour_df_all['count']

# sort the hour by 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4
dep_hour_df_all['recording_hour'] = dep_hour_df_all['hour'].apply(recording_index)
dep_hour_df_all['recording_hour'] = dep_hour_df_all['recording_hour'] - dep_hour_df_all['recording_hour'].min() + 1 # rescale

dep_hour_df_all = dep_hour_df_all.sort_values(by=['recording_hour', 'order_name'], ascending=[True, True])
dep_hour_df_all

In [ ]:
dep_hour_df_all['adjust'].value_counts()

In [ ]:
order_names = dep_hour_df_all['order_name'].unique()
palette = sns.color_palette("tab20", len(order_names))
order_colors = dict(zip(order_names, palette))

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(10, 10))

for order, group in dep_hour_df_all.groupby('order_name'):
    df_loop = pd.concat([group, group.iloc[[0]]]).reset_index(drop=True)
    df_loop = df_loop.sort_values(by='recording_hour').reset_index(drop=True)

    # add a discontinuity between the first and last point
    if any(df_loop['hour'] < 12):
        end_of_session = df_loop.loc[df_loop['hour'] < 12, 'hour'].max()
        end_of_session_index = df_loop.loc[df_loop['hour'] == end_of_session].index[0]
        extra_row = pd.DataFrame({'hour': end_of_session, 'count': [np.nan], 'order_name': [order]})

        # add a row after the end_of_session hour
        if end_of_session_index < len(df_loop) - 1:
            next_hour = df_loop.loc[end_of_session_index + 1, 'hour']
            df_loop = pd.concat([df_loop.iloc[:end_of_session_index + 1], extra_row, df_loop.iloc[end_of_session_index + 1:]], ignore_index=True)
        else:
            df_loop = pd.concat([df_loop, extra_row], ignore_index=True)

    theta = np.deg2rad(df_loop['hour'] * 15)  # 24h clock mapped to 360°

    # Plot main line
    line = ax.plot(theta, df_loop['adjusted_count'], label=order, color=order_colors[order])[0]

    # Make first and last segments dashed if they exist
    mask = df_loop['recording_hour'] < 3
    ax.plot(theta[mask],
            df_loop.loc[mask, 'adjusted_count'],
           color='white', linestyle='--', linewidth=line.get_linewidth())

    mask = df_loop['recording_hour'] > dep_hour_df_all['recording_hour'].max() - 2
    ax.plot(theta[mask],
            df_loop.loc[mask, 'adjusted_count'],
           color='white', linestyle='--', linewidth=line.get_linewidth())

ax.set_theta_direction(-1)
ax.set_theta_offset(np.pi / 2)
ax.set_xticks(np.linspace(0, 2 * np.pi, 24, endpoint=False))
ax.set_xticklabels([f"{h}:00" for h in range(24)])
ax.set_yscale('log')

# add legend for dashed line
ax.plot([], [], color='white', linestyle='-', linewidth=2, label=' ')
ax.plot([], [], color='black', linestyle='--', linewidth=2, label='Full hour projection')
ax.legend(loc='upper left', bbox_to_anchor=(1.2, 1), title='Order Name', fontsize=10)

plt.title('Average Activity by Hour of Day\n\n')
# plt.show()

# save the polar plot
fig.savefig(f"{plot_dir}/moth_activity_polar_plot.png", bbox_inches='tight', dpi=300)

In [ ]:
# subplots with activity plots for each deployment
ncols=2
nrows=int(math.ceil(len(deployments)/ncols))
fig, axs = plt.subplots(nrows, ncols, figsize=(12, 24), sharex='all', subplot_kw={'projection': 'polar'})
axs = axs.flatten()

for i, dep in enumerate(dep_df['dep'].unique()):
    df = dep_hour_df.loc[dep_hour_df['dep'] == dep, ]

    for order, group in df.groupby('order_name'):

        df_loop = pd.concat([group, group.iloc[[0]]]).reset_index(drop=True)
        df_loop['recording_hour'] = df_loop['hour'].apply(recording_index)
        df_loop['recording_hour'] = df_loop['recording_hour'] - df_loop['recording_hour'].min() + 1 # rescale
        df_loop = df_loop.sort_values(by='recording_hour').reset_index(drop=True)

        # after the maximum hour under 12, add a discontinuity
        if any(df_loop['hour'] < 12):
            end_of_session = df_loop.loc[df_loop['hour'] < 12, 'hour'].max()
            end_of_session_index = df_loop.loc[df_loop['hour'] == end_of_session].index[0]
            # Create extra_row with all necessary columns including adjusted_count
            extra_row = pd.DataFrame({
                'dep': [dep],
                'hour': [end_of_session],
                'count': [np.nan],
                'adjusted_count': [np.nan],  # Add this column
                'order_name': [order],
                'was_adjusted': [False]
            })

            # add a row after the end_of_session hour
            if end_of_session_index < len(df_loop) - 1:
                # get the next hour
                next_hour = df_loop.loc[end_of_session_index + 1, 'hour']
                df_loop = pd.concat([df_loop.iloc[:end_of_session_index + 1], extra_row, df_loop.iloc[end_of_session_index + 1:]], ignore_index=True)
            else:
                df_loop = pd.concat([df_loop, extra_row], ignore_index=True)

        # Calculate was_adjusted after all concatenations are done, and handle NaN values
        df_loop['was_adjusted'] = (df_loop['adjusted_count'] != df_loop['count']).fillna(False)

        theta = np.deg2rad(df_loop['hour'] * 15)  # 24h clock mapped to 360°
        line = axs[i].plot(theta, df_loop['adjusted_count'], label=order, color=order_colors[order])

        # Make first and last segments dashed if they exist
        mask = (df_loop['was_adjusted']) & (df_loop['hour'] > 12)
        axs[i].plot(theta[mask], df_loop.loc[mask, 'adjusted_count'],
            color='white', linestyle='--', linewidth=line[0].get_linewidth())

        mask = (df_loop['was_adjusted']) & (df_loop['hour'] < 12)
        axs[i].plot(theta[mask], df_loop.loc[mask, 'adjusted_count'],
            color='white', linestyle='--', linewidth=line[0].get_linewidth())


    axs[i].set_theta_direction(-1)
    axs[i].set_theta_offset(np.pi / 2)
    axs[i].set_xticks(np.linspace(0, 2 * np.pi, 24, endpoint=False))
    axs[i].set_xticklabels([f"{h}:00" for h in range(24)])
    axs[i].set_yscale('log')
    axs[i].set_title(f'{dep}\n\n')

# Create first legend for colors (order names)
color_handles = [plt.Line2D([0], [0], color=color, label=order, linewidth=2)
                for order, color in order_colors.items()]
color_legend = fig.legend(handles=color_handles, title='Order Name', loc='lower center',
                         ncol=3, bbox_to_anchor=(0.5, -0.12))

# Create second legend for linestyle (discontinuity)
style_handles = [plt.Line2D([0], [0], color='black', linestyle='-', linewidth=2, label='Recorded Hours'),
                plt.Line2D([0], [0], color='black', linestyle='--', linewidth=2, label='Hour Projection')]
style_legend = fig.legend(handles=style_handles, title='Line Type', loc='lower center',
                         ncol=2, bbox_to_anchor=(0.5, -0.18))

plt.suptitle('Average Insect Activity by Hour of Day\n\n')
plt.tight_layout(pad=3.0)
plt.subplots_adjust(top=0.93, bottom=0.25)  # Increased bottom margin to accommodate both legends
# plt.show()

fig.savefig(f"{plot_dir}/moth_activity_polar_plot_by_deployment.png", bbox_inches='tight', dpi=300)

## Line Plots

In [ ]:
# subplots with activity plots for each deployment
ncols=3
nrows=int(math.ceil(len(deployments)/ncols))
fig, axs = plt.subplots(nrows, ncols, figsize=(12, 12))
axs = axs.flatten()

for i, dep in enumerate(dep_df['dep'].unique()):
    df = dep_hour_df.loc[dep_hour_df['dep'] == dep, ]
    # If the hour is < 12 then add 24 to the hour
    df.loc[df['hour'] < 12, 'hour'] = df.loc[df['hour'] < 12, 'hour'] +24 # df['hour'].apply(lambda x: x + 24 if x < 12 else x)

    for order, group in df.groupby('order_name'):
        group = group.sort_values('hour')
        axs[i].plot(group['hour'], group['adjusted_count'], label=order, color=order_colors[order])
    axs[i].set_yscale('log')
    axs[i].set_title(f'{dep}')
    axs[i].set_ylabel('Count (log scale)')
    axs[i].set_xlabel('Hour of Day')

    min_hour = df['hour'].min()
    max_hour = df['hour'].max()
    times_raw = np.arange(16, 4+24 + 1, 1)
    times_labels = [x - 24 if x > 24 else x for x in times_raw]
    times_labels = [f"{int(x)}:00" for x in times_labels]
    axs[i].set_xticks(times_raw)
    axs[i].set_xlim(16, 4+24+1)
    axs[i].set_xticklabels(times_labels, rotation=-45, ha='left')


# Build a single legend using the shared color map
handles = [plt.Line2D([0], [0], color=color, label=order) for order, color in order_colors.items()]
fig.legend(handles=handles, title='Order Name', loc='lower center', ncol=3, bbox_to_anchor=(0.5, -0.2))

plt.suptitle('Insect Activity by Hour of Day\n\n')
plt.tight_layout()
plt.subplots_adjust(top=0.93)
fig.savefig(f"{plot_dir}/moth_activity_line_plot_by_deployment.png", bbox_inches='tight', dpi=300)
# plt.show()

# Violin Plots

Average number of crops per night per deployment

In [ ]:
dep_hour_sub = dep_hour[dep_hour['count'] == dep_hour['adjusted_count']]
dep_hour_sub

In [ ]:
# subplot for each order, with a violin plot of the deployment on the x-axis and the count on the y-axis
order_names = sorted(dep_hour_sub['order_name'].unique())
palette = sns.color_palette("tab20", len(order_names))
order_colors = dict(zip(order_names, palette))

fig, axs = plt.subplots(math.ceil(len(order_names)/3), 3, figsize=(18, len(order_names)), sharex=True)
axs = axs.flatten()

for i, order in enumerate(order_names):
    df = dep_hour_sub[dep_hour_sub['order_name'] == order]
    sns.violinplot(x='dep', y='count', data=df, ax=axs[i], density_norm='width', color=order_colors[order], inner='quartile')
    axs[i].set_title(f'{order} Count by Deployment')
    axs[i].set_ylabel('Mean Count')
    axs[i].set_xlabel('')
    axs[i].set_xticks(range(len(dep_hour_sub['dep'].unique())))
    axs[i].set_xticklabels(dep_hour_sub['dep'].unique(), rotation=45, ha='right')
    axs[i].grid(True, which='both', linestyle='--', linewidth=0.5)
    # axs[i].set_ylim(1, df['count'].max() * 1.1)
    #

# make the last plot empty
if len(order_names) % 3 != 0:
    axs[len(order_names)].axis('off')

plt.suptitle('Distribution of Insect Counts per Night by Deployment and Order Name\n\n')
plt.tight_layout()

fig.savefig(f"{plot_dir}/count_violin_plot_by_deployment.png", bbox_inches='tight', dpi=300)
plt.show()

# Crop counts

In [ ]:
dep_df

In [ ]:
plt.figure(figsize=(3, 3))
sns.barplot(data=dep_df.sort_values('hourly_count', ascending=False), hue='hourly_count', y='hourly_count', x='dep')
plt.title('All crops per deployment', size=8)
plt.xticks(rotation=45, ha='right', fontsize=6)
plt.tight_layout()
# plt.yscale('log')
plt.yticks(fontsize=6)
plt.legend().set_visible(False)
plt.ylabel('Mean Nightly Count', size=8)
plt.xlabel('')

# save
plt.savefig(f"{plot_dir}/all_crops_per_deployment.png", bbox_inches='tight', dpi=300)
plt.show()

### Moth Activity

In [ ]:
df = moth_only_df(inference_csvs)

In [ ]:
df['image_path'].value_counts().plot(kind='hist', bins=100, figsize=(5, 3))
plt.title('Moth crops per image')
plt.xlabel('Number of moth crops per image (n > 0)')
plt.yscale('log')
plt.savefig(f"{plot_dir}/moths_per_image.png", bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
df['dep'].unique()

In [ ]:
# Show the confidence distribution for each deployment in subplots
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(10, 5), sharex=True)
axes = axes.flatten()

for i, dep in enumerate(df['dep'].unique()):
    ax = axes[i]
    dep_df = df.loc[df['dep'] == dep, ]
    dep_df['image_path'].value_counts().plot(kind='hist', bins=100, ax=ax)
    ax.set_title(dep)
    ax.set_xlabel('Number of moth crops per image (n > 0)')
    ax.set_yscale('log')

plt.suptitle(f'Confidence Distribution of 5 Most Likely Species Predictions')
plt.tight_layout()
plt.savefig(f"{plot_dir}/confidence_distribution.png", bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
dep_counts = df['dep'].value_counts()
dep_counts = dep_counts.reset_index()
dep_counts.columns = ['deployment', 'count']

plt.figure(figsize=(7, 5))
sns.barplot(data=dep_counts, hue='count', y='count', x='deployment')
plt.title('Moth crops per deployment')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.yscale('log')
# plt.legend(visible=False)
plt.savefig(f"{plot_dir}/moth_crops.png", bbox_inches='tight', dpi=300)
plt.show()

## Order Prediction Plots

In [ ]:
# plot showing the correlation between order confidence and crop area
# this plot takes a while to compile, uncomment if you want to run
# sns.regplot(x=df['order_confidence'], y=df['crop_area'], logx=True, line_kws=dict(color="r"))
# plt.yscale('log')
# plt.show()

In [ ]:
order_counts = cat_summary(inference_csvs)

In [ ]:
order_counts.head()

In [ ]:
plt.figure(figsize=(7, 5))
sns.barplot(data=order_counts, hue='deployment', y='count', x='order_name')
plt.title('Counts by Deployment and Order')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Order')
plt.ylabel('Number of Crops')
plt.legend(title='Order Name', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## Species Prediction Plots (Macro only)

In [ ]:
species_counts = cat_summary(inference_csvs, category='top_1_species', macro_only=True)

In [ ]:
# subset to only the top 10 species
top_species = species_counts[['top_1_species',  'count']].groupby('top_1_species').sum().reset_index()
top_species = top_species.sort_values(by='count', ascending=False)
top_species = top_species.head(10)

In [ ]:
species_counts

In [ ]:
# match the order_name fromd

In [ ]:
plt.figure(figsize=(7, 5))
sns.barplot(data=top_species, y='count', x='top_1_species')
plt.title('Top Macro Species Counts')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Most Likely Species')
plt.ylabel('Number of Crops')
plt.tight_layout()

# save the plot
plt.savefig(f"{plot_dir}/top_species_counts.png", bbox_inches='tight', dpi=300)

plt.show()

In [ ]:
top_species.head()

In [ ]:
species_counts['deployment'].unique()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=math.ceil(len(species_counts['deployment'].unique()) / 2), figsize=(10, 7), sharey=True)
axes = axes.flatten()

for i, dep in enumerate(species_counts['deployment'].unique()):
    ax = axes[i]
    dep_df = species_counts.loc[species_counts['deployment'] == dep, ]
    dep_df = dep_df.sort_values(by='count', ascending=False).head(10)
    sns.barplot(data=dep_df, y='count', x='top_1_species', ax=ax)
    ax.set_title(dep)
    ax.set_xticks(range(len(dep_df)))
    ax.set_xticklabels(dep_df['top_1_species'], rotation=45, ha='right')
    ax.set_xlabel('')
    ax.set_ylabel('Number of Crops')

plt.suptitle('Top Species Observations by Deployment')
plt.tight_layout()
plt.savefig(f"{plot_dir}/top_species_counts_by_deploument.png", bbox_inches='tight', dpi=300)

plt.show()


In [ ]:
plt.figure(figsize=(7, 5))
sns.barplot(data=species_counts.loc[species_counts['top_1_species'].isin(top_species['top_1_species']), ], hue='deployment', y='count', x='top_1_species')
plt.title('Top Species Counts by Deployment')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Most popular species', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.xlabel('Most Likely Species')
plt.ylabel('Number of Crops')
plt.show()

In [ ]:
df['top_1_confidence'].plot(kind='hist', bins=50, figsize=(6, 3))
df['top_2_confidence'].plot(kind='hist', bins=50, color='orange', alpha=0.5)
df['top_3_confidence'].plot(kind='hist', bins=50, color='yellow', alpha=0.5)
df['top_4_confidence'].plot(kind='hist', bins=50, color='green', alpha=0.5)
df['top_5_confidence'].plot(kind='hist', bins=50, color='purple', alpha=0.5)

plt.legend(['Top 1 Confidence', 'Top 2 Confidence', 'Top 3 Confidence', 'Top 4 Confidence', 'Top 5 Confidence'])
plt.title('Confidence Distribution of 5 Most Likely Species Predictions')
plt.xlabel('Confidence')
plt.savefig(f"{plot_dir}/confidence_distribution.png", bbox_inches='tight', dpi=300)

plt.show()

In [ ]:
# Show the confidence distribution for each deployment in subplots
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(10, 5), sharex=True)
axes = axes.flatten()

for i, dep in enumerate(df['dep'].unique()):
    ax = axes[i]
    dep_df = df.loc[df['dep'] == dep, ]
    dep_df['top_1_confidence'].plot(kind='hist', bins=50, ax=ax, alpha=0.5)
    dep_df['top_2_confidence'].plot(kind='hist', bins=50, ax=ax, color='orange', alpha=0.5)
    dep_df['top_3_confidence'].plot(kind='hist', bins=50, ax=ax, color='yellow', alpha=0.5)
    dep_df['top_4_confidence'].plot(kind='hist', bins=50, ax=ax, color='green', alpha=0.5)
    dep_df['top_5_confidence'].plot(kind='hist', bins=50, ax=ax, color='purple', alpha=0.5)
    ax.legend().set_visible(False)
    ax.set_title(dep)
    ax.set_xlabel('Confidence')
    ax.set_ylabel('Count')


handles, _ = axes[0].get_legend_handles_labels()
labels = ['Top 1 Confidence', 'Top 2 Confidence', 'Top 3 Confidence', 'Top 4 Confidence', 'Top 5 Confidence']
fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(1.2, 0.9))
plt.suptitle(f'Confidence Distribution of 5 Most Likely Species Predictions')
plt.tight_layout()
plt.savefig(f"{plot_dir}/confidence_distribution_by_deployment.png", bbox_inches='tight', dpi=300)
plt.show()

# Crop examples

## Most Confident cases for each order

In [ ]:
def load_df_from_file(inference_csvs, sort_by='top_1_confidence', ascending=False, n_keep=100, group_by_col=None, remove_none=True):
    """
    Load the dataframe from the CSV file and sort it by the specified column and keep the top n entries (with an optional group by).

    Args:
        inference_csvs (list): List of CSV file paths to read.
        sort_by (str): The column name to sort by.
        ascending (bool): Whether to sort in ascending order.
        n_keep (int): The number of entries to keep from each file.
        group_by_col (str): The column name to group by before keeping the top n entries.
        remove_none (bool): Whether to remove entries with 'NO DETECTIONS FOR IMAGE'.

    Returns:
        pd.DataFrame: The sorted dataframe.
    """
    df = pd.DataFrame()
    for c in tqdm(inference_csvs):
        try:
            input_df = pd.read_csv(c, low_memory=False)
        except Exception as e:
            print(f" - Error reading {c}: {e}")
            continue
        if remove_none:
            input_df = input_df.loc[input_df['crop_status'] != 'NO DETECTIONS FOR IMAGE', ]
        input_df = input_df.drop_duplicates(subset=[ 'x_min', 'x_max', 'y_min', 'y_max'])
        input_df = input_df.sort_values(by=sort_by, ascending=ascending)

        input_df['dep'] = os.path.basename(c).split('.')[0].split('_')[0]
        input_df['crop_area'] = (input_df['x_max'] - input_df['x_min']) * (input_df['y_max'] - input_df['y_min'])

        # set new keys column as 'dep' and 'image_path' combined
        input_df['keys'] = input_df['image_path'].apply(lambda x: f"{input_df['dep'].iloc[0]}/snapshot_images/{os.path.basename(x)}")

        if group_by_col is not None:
            input_df = input_df.groupby(group_by_col).head(n_keep)
        else:
            input_df = input_df.head(n_keep)
        df = pd.concat([df, input_df])
        del input_df

    df = df.sort_values(by=['dep', sort_by], ascending=[True, ascending]).reset_index(drop=True)

    return df

In [ ]:
order_df = load_df_from_file(inference_csvs, sort_by='order_confidence', ascending=False, n_keep=100, remove_none=True, group_by_col='order_name')

# get *the* most confident image for each order and deployment
order_df = order_df.groupby(['order_name', 'dep']).head(1).reset_index(drop=True)

In [ ]:
# order by confidence and subset
df_order = order_df.sort_values(by='order_confidence', ascending=False)
df_order = df_order.loc[df_order['order_confidence'] > 0.99, ]
df_order.reset_index(drop=True, inplace=True)

print(df_order.shape)

In [ ]:
download_dir

In [ ]:
# download the relevant images
os.makedirs(os.path.join(download_dir, 'order'), exist_ok=True)

for i, row in df_order.iterrows():
    download_images(client, transfer_config, row['keys'], os.path.join(download_dir, 'order'), region)

In [ ]:
from PIL import Image
from PIL import ImageDraw, Image, ImageFont

In [ ]:
def annotate_image2(image_path, dir, df, ax, scaling_required=False, crop_to_highlight=None, buffer=5, subtitle=None):
    df_image = df.loc[df['image_path'] == image_path, ]

    img = plt.imread(f"{dir}/{os.path.basename(image_path)}")
    image = Image.open(f"{dir}/{os.path.basename(image_path)}").convert("RGB")
    subp = ax.imshow(img, origin='lower')

    draw = ImageDraw.Draw(image)

    for j, row in df_image.iterrows():
        x_min = float(row['x_min']) -buffer
        y_min = float(row['y_min']) -buffer
        x_max = float(row['x_max']) +buffer
        y_max = float(row['y_max']) +buffer

        if scaling_required:
            original_width, original_height = image.size
            x_min = x_min /300 * original_width
            y_min = y_min /300 * original_height
            x_max = x_max /300 * original_width
            y_max = y_max /300 * original_height

        x = float(x_min)
        y = float(y_min)
        w = float(x_max - x_min)
        h = float(y_max - y_min)

        if (row['class_name'] == "moth") and ('Lepidoptera' not in row['order_name']):
            col = 'orange'
        elif (row['class_name'] != "moth") and ('Lepidoptera' in row['order_name']):
            col = 'purple'
        elif (row['class_name'] == "moth") and ('Lepidoptera' in row['order_name']):
                col = 'green'
        else:
            col = 'red'

        alph = 1
        if crop_to_highlight is not None:
            if row['crop_status'] != crop_to_highlight:
                alph = 0.2

        try:
            if os.path.exists("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"):
                font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
            else:
                import cv2

                font_path = os.path.join(
                    cv2.__path__[0], "qt", "fonts", "DejaVuSans.ttf"
                )
            font = ImageFont.truetype(font_path, size=50)
        except Exception as e:
            print(f"Loading default font, could not another: {e}")
            font = ImageFont.load_default()


        if (row['class_name'] == "moth") or ("Lepidoptera" in row['order_name']):
            draw.text(
                (x_min, y_min),
                f"{row['top_1_species']}: {row['top_1_confidence']:.2f}",
                fill=col,
                font=font,
            )

        draw.rectangle([x_min, y_min, x_max, y_max], outline=col, width=3)

        # remove axes
        ax.axis('off')

        if subtitle is not None:
            ax.set_title(subtitle, fontsize=16, pad=20)

        rect = plt.Rectangle((x, y), w, h, fill=False, edgecolor=col, linewidth=1, alpha=alph)
        ax.add_patch(rect)

    return img

In [ ]:
df_order

In [ ]:
# for each unique image, annotate the bounding box
fig, ax = plt.subplots(13, 3, figsize=(10, 17))
ax = ax.ravel()

df_order = df_order.sort_values(by='order_name', ascending=False)

# Use enumerate to get the correct loop index for ax[]
for j, (_, row) in enumerate(df_order.iterrows()):
    annotate_image2(
        image_path=row['image_path'],
        dir=os.path.join(download_dir, 'order'),
        df=df_order,
        ax=ax[j],
        crop_to_highlight=row['crop_status'],
        buffer=20,
        scaling_required=False,
        subtitle=f"{row['dep']}\n{row['order_name']}, {row['order_confidence']:.2f}"
    )

# Turn off any unused axes
for i in range(len(df_order), len(ax)):
    ax[i].axis('off')

plt.tight_layout()
plt.savefig(f"{plot_dir}/order_plots.png", bbox_inches='tight', dpi=300)
plt.show()


## Most Confident Species Predictions

In [ ]:
df_confident = load_df_from_file(inference_csvs, sort_by='top_1_confidence', ascending=False, n_keep=10, remove_none=True, group_by_col='top_1_species')

In [ ]:
df_confident_backup = df_confident.copy()

In [ ]:
# order by top_1_confidence
# df_confident = df.sort_values(by='top_1_confidence', ascending=False)
# df_confident.reset_index(drop=True, inplace=True)

# # remove duplicated rows by image path, and bounding box
# df_confident = df_confident.drop_duplicates(subset=['image_path', 'x_min', 'y_min', 'x_max', 'y_max'])
# df_confident = df_confident.drop_duplicates(subset=['top_1_species'])

top_n=20

df_confident = df_confident.head(top_n)

In [ ]:
# sort by image_path
df_confident.sort_values(by='image_path', inplace=True)
df_confident.reset_index(drop=True, inplace=True)
df_confident

In [ ]:
df_confident.columns

In [ ]:
# download the relevant images
os.makedirs(os.path.join(download_dir, 'confident'), exist_ok=True)

for i, row in df_confident.head(top_n).iterrows():
    download_images(client, transfer_config, row['keys'], os.path.join(download_dir, 'confident'), region)

In [ ]:
df_confident.shape

In [ ]:
# for each unique image, annotate the bounding box
fig, ax = plt.subplots(4, 5, figsize=(13.33, 7.5))
ax = ax.ravel()

df_confident = df_confident.sort_values(by='top_1_confidence', ascending=False)

# for eah row in df_confident, get the image_path
for i, row in df_confident.iterrows():
    annotate_image2(
        row['image_path'],
        os.path.join(download_dir, 'confident'),
        df_confident,
        ax[i],
        crop_to_highlight=row['crop_status'],
        buffer=20,
        scaling_required=False,
        subtitle=f"{row['dep']}\n{row['top_1_species']}, {row['top_1_confidence']:.2f}"
    )

for i in range(len(df_confident), len(ax)):
    ax[i].axis('off')

plt.tight_layout()
plt.show()

## Largest Moths

In [ ]:
df_area = load_df_from_file(inference_csvs, sort_by='crop_area', ascending=False, n_keep=10, remove_none=True, group_by_col='order_name')

In [ ]:
df_area_backup = df_area.copy()

### First look at large moths

In [ ]:
# order by moth size
df_area = df_area_backup.sort_values(by='crop_area', ascending=False)
df_area.reset_index(drop=True, inplace=True)
df_area = df_area.drop_duplicates(subset=['top_1_species'])


top_n=20

df_area = df_area.head(top_n)

df_area.reset_index(drop=True, inplace=True)

In [ ]:
# download the relevant images
os.makedirs(os.path.join(download_dir, 'largest'), exist_ok=True)

for i, row in df_area.iterrows():
    download_images(client, transfer_config, row['keys'], os.path.join(download_dir, 'largest'), region)

In [ ]:
# for each unique image, annotate the bounding box
fig, ax = plt.subplots(4, 5, figsize=(13.33, 7.5))
ax = ax.ravel()

df_area = df_area.sort_values(by='crop_area', ascending=False)

# for eah row in df_confident, get the image_path
for i, row in df_area.iterrows():
    annotate_image2(
        row['image_path'],
        os.path.join(download_dir, 'largest'),
        df_area,
        ax[i],
        crop_to_highlight=row['crop_status'],
        buffer=20,
        scaling_required=False,
        subtitle=f"{row['top_1_species']}, ({row['top_1_confidence']:.2f})"
    )

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(13.33, 7.5))
# ax = ax.ravel()

df_area = df_area.sort_values(by='crop_area', ascending=False)

# for eah row in df_confident, get the image_path
for i, row in df_area.iloc[1:2, ].iterrows():
    annotate_image2(
        row['image_path'],
        os.path.join(download_dir, 'largest'),
        df_area,
        ax,
        crop_to_highlight=row['crop_status'],
        buffer=20,
        scaling_required=False,
        subtitle=f"{row['top_1_species']}, ({row['top_1_confidence']:.2f})"
    )

plt.tight_layout()
plt.show()

### Look at large non-moths

In [ ]:
# order by moth size
df_area = df_area_backup.sort_values(by='crop_area', ascending=False)
df_area.reset_index(drop=True, inplace=True)


df_area = df_area.loc[df_area['order_name'] != 'Lepidoptera Macros', ]
df_area = df_area.loc[df_area['order_name'] != 'Lepidoptera Micros', ]
df_area = df_area.loc[df_area['class_name'] != 'moth', ]

top_n=20

df_area = df_area.head(top_n)

df_area.reset_index(drop=True, inplace=True)

In [ ]:
# download the relevant images
os.makedirs(os.path.join(download_dir, 'largest_nm'), exist_ok=True)

for i, row in df_area.iterrows():
    download_images(client, transfer_config, row['keys'], os.path.join(download_dir, 'largest_nm'), region)

In [ ]:
# for each unique image, annotate the bounding box
fig, ax = plt.subplots(4, 5, figsize=(13.33, 7.5))
ax = ax.ravel()

df_area = df_area.sort_values(by='crop_area', ascending=False)

# for eah row in df_confident, get the image_path
for i, row in df_area.iterrows():
    annotate_image2(
        row['image_path'],
        os.path.join(download_dir, 'largest_nm'),
        df_area,
        ax[i],
        crop_to_highlight=row['crop_status'],
        buffer=20,
        scaling_required=False,
        subtitle=f"{row['top_1_species']}, ({row['top_1_confidence']:.2f})"
    )

plt.tight_layout()
plt.show()

In [ ]:
df_order.head()

## Busiest Images

In [ ]:
# from each inference_script subset to the most common 5 images
cutoff=100
all_paths = pd.DataFrame()

for x in tqdm(inference_csvs):
    try:
        df = pd.read_csv(x, low_memory=False)
    except Exception as e:
        print(f" - Error reading {x}: {e}")
        continue
    df = df.loc[df['crop_status'] != 'no detections for image.', ]
    df = df.drop_duplicates(subset=['x_min', 'x_max', 'y_min', 'y_max'])

    # get the most popular image_path
    paths = df['image_path'].value_counts()
    paths = paths[paths > cutoff].index.tolist()

    if len(paths) > 5:
        paths = paths[:5]

    df = df.loc[df['image_path'].isin(paths), ]
    df['dep'] = os.path.basename(x).split('.')[0].split('_')[0]

        # set new keys column as 'dep' and 'image_path' combined
    df['keys'] = df['image_path'].apply(lambda x: f"{df['dep'].iloc[0]}/snapshot_images/{os.path.basename(x)}")

    all_paths = pd.concat([all_paths, df])

In [ ]:
all_paths.shape

# subset to the 20 most common paths
all_paths_df = all_paths.groupby('image_path').head(5).reset_index(drop=True)

all_paths_df.shape

In [ ]:
# download the relevant images
os.makedirs(os.path.join(download_dir, 'busy'), exist_ok=True)

for i, row in all_paths_df.iterrows():
    download_images(client, transfer_config, row['keys'], os.path.join(download_dir, 'busy'), region)

## Blurriest Crops

In [ ]:
from amber_inferences.utils.inference_scripts import download_image_from_key

In [ ]:
# order by moth size
df_blur = df.sort_values(by='crop_bluriness', ascending=False)
df_blur.reset_index(drop=True, inplace=True)

df_blur = df_blur.drop_duplicates(subset=['top_1_species'])
top_n=20

df_blur = df_blur.head(top_n)

df_blur.reset_index(drop=True, inplace=True)

In [ ]:
download_dir = '/home/users/katriona/amber-inferences/data/qc_plots/singapore'

In [ ]:
region

In [ ]:
# download the relevant images
os.makedirs(os.path.join(download_dir, 'blur'), exist_ok=True)

for i, row in df_blur.iterrows():
    print(row['keys'])
    download_image_from_key(client, row['keys'], region, os.path.join(download_dir, 'blur'))
    # download_images(client, transfer_config, row['keys'], os.path.join(download_dir, 'blur'), region)

In [ ]:
df_blur

In [ ]:
# for each unique image, annotate the bounding box
fig, ax = plt.subplots(4, 5, figsize=(13.33, 7.5))
ax = ax.ravel()

df_blur = df_blur.sort_values(by='crop_bluriness', ascending=False)

# for eah row in df_blur, get the image_path
for i, row in df_blur.iterrows():
    annotate_image(
        row['image_path'],
        os.path.join(download_dir, 'blur'),
        df_blur,
        ax[i],
        crop_to_highlight=row['crop_status'],
        buffer=20,
        scaling_required=False,
        subtitle=f"Crop blur: {float(row['crop_bluriness']):.3f}\nImage blur: {float(row['image_bluriness']):.3f}"
    )

plt.tight_layout()
plt.show()

## Bluriest Images

In [ ]:
df.head()

In [ ]:
df_eco = df.loc[df['dep'] == "ECOLINK", ]
# df_eco = df.loc[df['image_bluriness'] < 50, ]

df_eco = df_eco.drop_duplicates(subset=['image_path'])

df_eco['date'] = pd.to_datetime(df_eco['image_datetime'])
df_eco['date'] = df_eco['date'].dt.strftime('%Y-%m-%d')

# plot date vs image_bluriness
plt.figure(figsize=(10, 5))

sns.scatterplot(data=df_eco, x='date', y='image_bluriness', alpha=0.7)
plt.xticks(rotation=45, ha='right')
plt.title('Image Bluriness by Date for Ecolink Deployment')
plt.show()


In [ ]:
# for each deployment print the average image bluriness
dep_bluriness = df.drop_duplicates('image_path').groupby('dep')['image_bluriness'].mean().reset_index()

print(dep_bluriness)

In [ ]:
(df_eco.loc['image_bluriness'] > 50).value_counts()

In [ ]:
# download the relevant images
os.makedirs(os.path.join(download_dir, 'ecolink2'), exist_ok=True)

for i, row in df_eco.loc[df_eco['image_bluriness'] < 50, ].sort_values('image_bluriness').head().iterrows():
    print(row['keys'])
    download_image_from_key(client, row['keys'], region, os.path.join(download_dir, 'ecolink2'))

In [ ]:
# subplots for each deployment, plot crop_blurriness vs top_1_confidence
fig, ax = plt.subplots(2, 5, figsize=(10, 5), sharex=True, sharey=True)
ax = ax.flatten()

for i, dep in enumerate(df['dep'].unique()):
    dep_df = df.loc[df['dep'] == dep, ]
    sns.scatterplot(data=dep_df, x='crop_bluriness', y='top_1_confidence', ax=ax[i], alpha=0.5)
    ax[i].set_title(dep)
    ax[i].set_xlabel('Crop Blur')
    ax[i].set_ylabel('Top 1 Confidence')
plt.suptitle('Crop Blur vs Top 1 Confidence by Deployment')
plt.tight_layout()
plt.savefig(f"{plot_dir}/crop_blur_vs_top_1_confidence_by_deployment.png", bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
df.head()

In [ ]:
# subplots by deployment, plot a histogram of image blurriness
fig, ax = plt.subplots(2, 5, figsize=(10, 5), sharex=True, sharey=True)
ax = ax.flatten()

for i, dep in enumerate(df['dep'].unique()):
    dep_df_sub = df.loc[df['dep'] == dep, ]
    # drop duplicated image
    dep_df_sub = dep_df_sub.drop_duplicates(subset=['image_path'])
    sns.histplot(data=dep_df_sub, x='image_bluriness', ax=ax[i], bins=50, kde=True)
    ax[i].set_title(dep)
    ax[i].set_xlabel('Image Blur')
    ax[i].set_ylabel('Count')
    # ax[i].set_xlim(0, 1)
plt.suptitle('Image Blur Distribution by Deployment')
plt.tight_layout()
plt.savefig(f"{plot_dir}/image_blur_distribution_by_deployment.png", bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# order by moth size
df = df.astype({'image_bluriness': 'float'})
df_blur = df.sort_values(by='image_bluriness', ascending=False)

df_blur.reset_index(drop=True, inplace=True)

df_blur = df_blur.drop_duplicates(subset=['top_1_species'])
df_blur = df_blur.drop_duplicates(subset=['image_bluriness'])
top_n=20

df_blur = df_blur.head(top_n)

df_blur.reset_index(drop=True, inplace=True)
df_blur

In [ ]:
# download the relevant images
os.makedirs(os.path.join(download_dir, 'blur'), exist_ok=True)

for i, row in df_blur.iterrows():
    download_images(client, transfer_config, row['keys'], os.path.join(download_dir, 'blur'), region)

In [ ]:
# for each unique image, annotate the bounding box
fig, ax = plt.subplots(4, 5, figsize=(13.33, 7.5))
ax = ax.ravel()

df_blur = df_blur.sort_values(by='crop_bluriness', ascending=False)

# for eah row in df_blur, get the image_path
for i, row in df_blur.iterrows():
    annotate_image(
        row['image_path'],
        os.path.join(download_dir, 'blur'),
        df_blur,
        ax[i],
        crop_to_highlight=row['crop_status'],
        buffer=20,
        scaling_required=False,
        subtitle=f"Crop blur: {float(row['crop_bluriness']):.3f}\nImage blur: {float(row['image_bluriness']):.3f}"

    )

plt.tight_layout()
plt.show()

In [ ]:
import json
import pandas as pd

json_file = '/home/users/katriona/amber-inferences/keys/thailand_final/dep000074.json'

with open(json_file, "r") as f:
    chunks = json.load(f)


# create a dataframe from the json file as filename column
df_json = pd.DataFrame(chunks, columns=['filename'])


In [ ]:
df_json.head()

In [ ]:
df_json['datetime'] = df_json['filename'].apply(lambda x: x.split('/')[2].replace('-snapshot.jpg', '') )
df_json['datetime'] = pd.to_datetime(df_json['datetime'], format='%Y%m%d%H%M%S')

In [ ]:
# if the time is < 12, add 24 hours to the date
df_json['session'] = df_json['datetime']
df_json['session'] = df_json['datetime'].apply(lambda x: x - pd.Timedelta(days=1) if x.hour < 12 else x)
df_json['session'] = df_json['session'].dt.strftime('%Y-%m-%d')

In [ ]:
len(list(set(df_json['session'])))

In [ ]:
df_json

In [ ]:
# convert to a dict grouped by session
sessions = df_json.groupby('session')['filename'].apply(list).to_dict()

sessions

In [ ]:
sessions

In [ ]:
sessions['2024-07-24']

In [ ]:
sessions.keys()

# Tracking

In [ ]:
import pandas as pd

In [ ]:
csv_file = '/gws/nopw/j04/ceh_generic/kgoldmann/costarica_inferences_tracking/dep000031/dep000031_2024-06-03.csv'
df = pd.read_csv(csv_file)

In [ ]:
track_df = track_id_calc(df, cost_threshold=1)
print(f"Number of tracks for {os.path.basename(csv_file)}: {track_df['track_id'].nunique()}")

In [ ]:
df['image_path_basename'] = df['image_path'].apply(lambda x: os.path.basename(x))

df = df.merge(track_df, how='left', left_on=['image_path_basename', 'crop_status'], right_on=['image_path', 'crop_id'])
df = df.reset_index(drop=True)
df = df.loc[:, ~df.columns.str.contains('_y')]
df = df.rename(columns=lambda x: x.replace('_x', ''))

In [ ]:
df.to_csv('.test.csv', index=False)


In [ ]:
print(df['track_id'].value_counts(dropna=False))
df['track_id'].value_counts().plot(kind='hist', bins=100, figsize=(5, 3))

In [ ]:
print(df.shape)

df = df.loc[df['crop_status'] != 'No detections for this image.', ]

# drop nan
df = df.loc[df['best_match_crop'].notna(), ]
df = df.loc[df['total_cost'].notna(), ]

df.head()

In [ ]:
df['best_match_crop'].value_counts()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#histogram of total cost
df['total_cost'] = df['total_cost'].astype(float)
df['total_cost'].plot(kind='hist', bins=50, figsize=(10, 5))
plt.title('Total Cost Distribution')
plt.xlabel('Total Cost')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
def track_id_calc(best_matches, cost_threshold=1):
        # Make a copy and rename the relevant columns
    best_matches = best_matches.copy()

    best_matches = best_matches.loc[best_matches['total_cost'] != "", ]
    best_matches = best_matches.loc[best_matches['best_match_crop'].notna(), ]
    best_matches = best_matches.loc[best_matches['total_cost'].notna(), ]
    best_matches['image1'] = [os.path.basename(x) for x in best_matches['image_path']]
    best_matches['image2'] = best_matches['previous_image']
    best_matches['crop1'] = best_matches['crop_status']
    best_matches['crop2'] = best_matches['best_match_crop']

    # Filter based on the cost threshold
    filtered_matches = best_matches[best_matches["total_cost"] < cost_threshold]

    def node_id(image_path, crop_id):
        return f"{image_path}|{crop_id}"

    # Build a graph of matches under the threshold
    G_thresh = nx.Graph()
    for _, row in filtered_matches.iterrows():
        n1 = node_id(row['image1'], row['crop1'])
        n2 = node_id(row['image2'], row['crop2'])
        G_thresh.add_edge(n1, n2)

    # Assign track IDs from connected components
    track_mapping = {}
    for tid, component in enumerate(nx.connected_components(G_thresh)):
        for node in component:
            track_mapping[node] = f"Track_{str(tid).rjust(5, '0')}"

    # Collect all unique nodes from both sides
    all_nodes = set()
    for _, row in best_matches.iterrows():
        all_nodes.add(node_id(row['image1'], row['crop1']))
        all_nodes.add(node_id(row['image2'], row['crop2']))

    # Create lookup for cost (minimum per crop node)
    cost_lookup = {}
    for _, row in best_matches.iterrows():
        for prefix in ["1", "2"]:
            nid = node_id(row[f"image{prefix}"], row[f"crop{prefix}"])
            cost = row["total_cost"]
            cost_lookup[nid] = min(cost_lookup.get(nid, float("inf")), cost)

    # Assemble the final output rows
    output_rows = []
    for node in all_nodes:
        image_path, crop_id = node.rsplit("|", 2)
        output_rows.append({
            "image_path": image_path,
            "crop_id": crop_id,
            "track_id": track_mapping.get(node),  # May be None if unmatched under threshold
            "total_cost": cost_lookup.get(node)
        })

    output_df = pd.DataFrame(output_rows)

    # Assign unique track IDs to unmatched crops
    max_existing_id = (
        max(
            [int(tid.replace("Track_", "")) for tid in output_df["track_id"].dropna()],
            default=-1
        )
    )
    unmatched_mask = output_df["track_id"].isnull()
    unmatched_indices = output_df[unmatched_mask].index
    new_ids = [
        f"Track_{str(i).rjust(5, '0')}"
        for i in range(max_existing_id + 1, max_existing_id + 1 + unmatched_mask.sum())
    ]
    output_df.loc[unmatched_indices, "track_id"] = new_ids

    return output_df


In [ ]:
import math
import numpy as np
from torchvision import transforms
import torch
import pandas as pd
import networkx as nx
from itertools import product
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import os

In [ ]:
output_df = track_id_calc(df, cost_threshold=1)
output_df.head()

In [ ]:
# create a colour label
# Generate N unique colors for each track
num_tracks = output_df['track_id'].nunique()

# Use a colormap to get visually distinct colors
col_palette = 'tab20'  # You can choose any colormap from matplotlib
cmap = plt.cm.get_cmap(col_palette, num_tracks)

# Map track_id to hex colors
track_id_to_color = {
    track_id: mcolors.to_hex(cmap(i)) for i, track_id in enumerate(sorted(output_df['track_id'].unique()))
}

# Add color column to DataFrame
output_df['colour'] = output_df['track_id'].map(track_id_to_color)

In [ ]:
output_df['track_id'].value_counts().head()

In [ ]:
df['image_path_basename'] = df['image_path'].apply(lambda x: os.path.basename(x))

In [ ]:
df = df.merge(output_df, how='left', left_on=['image_path_basename', 'crop_status'], right_on=['image_path', 'crop_id'])
df = df.reset_index(drop=True)

In [ ]:
df.columns[df.columns.str.contains('_y')]

In [ ]:
# remove columns with '_y' suffix
df = df.loc[:, ~df.columns.str.contains('_y')]
# rename columns with '_x' suffix to remove it
df = df.rename(columns=lambda x: x.replace('_x', ''))

# drop the colour column
df = df.drop(columns=['colour'])

df.head()

In [ ]:
# save the df
output_file = './dep000031_2024-06-03_tracked.csv'
df.to_csv(output_file, index=False)

In [ ]:
# the number of snapshots per crop:
pd.DataFrame(df['track_id'].value_counts())['count'].value_counts().plot(kind='bar', figsize=(10, 5))

In [ ]:
df['track_id'].value_counts().head()

In [ ]:
# subset to the 3 most common track IDs
top_tracks = df['track_id'].value_counts().head(3).index.tolist()
df = df.loc[df['track_id'] == 'Track_00119', ]

In [ ]:
df.head()

In [ ]:
df['track_id'].value_counts()

In [ ]:
keys = 'dep000031/snapshot_images/' + df['image_path_basename'].unique()
print(len(keys))

In [ ]:
for i in range(len(keys)):
    try:
        download_images(client, transfer_config, keys[i], '/gws/nopw/j04/ceh_generic/kgoldmann/test', 'cri',)
    except Exception as e:
        print(f"Error downloading {keys[i]}: {e}")
        continue

In [ ]:
df_sub = df.loc[df['image_path_basename'].isin(os.listdir('/gws/nopw/j04/ceh_generic/kgoldmann/test')), ]

In [ ]:
# drop unique track_ids
frame_cutoff = 1

track_counts = df_sub["track_id"].value_counts()
valid_tracks = track_counts[track_counts > frame_cutoff].index
df_sub = df_sub[df_sub["track_id"].isin(valid_tracks)].reset_index(drop=True)
df_sub.head()

In [ ]:
df_sub.shape

In [ ]:
from amber_inferences.utils.plotting import *
from amber_inferences.utils.tracking import *

In [ ]:
df_sub['image_path_basename']

In [ ]:
imgs = []

for i, image_path in enumerate(df_sub['image_path_basename'].unique()):
    image_path_full = f'/gws/nopw/j04/ceh_generic/kgoldmann/test/{image_path}'
    imge = Image.open(image_path_full).convert("RGB")
    original_image = imge.copy()
    original_width, original_height = imge.size

    crops_df = df_sub.loc[df_sub['image_path_basename'] == os.path.basename(image_path_full), ]
    boxes = []
    if crops_df.shape[0] > 0:
        for j, row in crops_df.iterrows():
            boxes.append({
                'x_min': row['x_min'],
                'y_min': row['y_min'],
                'x_max': row['x_max'],
                'y_max': row['y_max'],
                'label': row['track_id'],
                'ann_col': 'black' #row['colour']
            })

    im = image_annotation(image_path_full, boxes=boxes, scale=False)
    out_path = f'/gws/nopw/j04/ceh_generic/kgoldmann/ann_test/{os.path.basename(image_path_full)}'
    im.save(out_path)